# Task 4.8

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 1

In [5]:
path= r"C:\Users\sosia\12-2024 Instacart Basket Analysis"

In [7]:
ords_prods_merge = pd.read_pickle(r'C:\Users\sosia\12-2024 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_merge1.pkl')

# 2

In [9]:
# Compute the mean of order_number grouped by department_id for the entire dataframe
department_order_means = ords_prods_merge.groupby('department_id')['order_number'].mean()

In [10]:
print(department_order_means)

department_id
1     14.800024
2     17.091743
3     17.913544
4     17.893092
5     15.214270
6     15.382135
7     17.694027
8     16.458105
9     15.957363
10    20.091818
11    16.482026
12    15.615061
13    16.484023
14    17.524632
15    15.691875
16    18.014071
17    16.150593
18    19.602850
19    17.631340
20    17.138607
21    21.956893
Name: order_number, dtype: float64


# 3

# **Key Takeaways**
- The full dataset provides a more precise and reliable analysis of customer ordering behavior across departments.
- The means are consistent in both outputs

# 4

In [17]:
# Calculate total orders per customer
ords_prods_merge['total_orders'] = ords_prods_merge.groupby('user_id')['order_number'].transform('max')

In [19]:
# Create the loyalty flag
ords_prods_merge.loc[ords_prods_merge['total_orders'] > 40, 'loyalty_flag'] = 'Loyal Customer'

In [21]:
ords_prods_merge.loc[(ords_prods_merge['total_orders'] > 10) & (ords_prods_merge['total_orders'] <= 40), 'loyalty_flag'] = 'Regular Customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['total_orders'] <= 10, 'loyalty_flag'] = 'New Customer'

In [25]:
# Display results
ords_prods_merge[['user_id', 'total_orders', 'loyalty_flag']].head(10)

,user_id,total_orders,loyalty_flag
0,1,10,New Customer
1,1,10,New Customer
2,1,10,New Customer
3,1,10,New Customer
4,1,10,New Customer
5,1,10,New Customer
6,1,10,New Customer
7,1,10,New Customer
8,1,10,New Customer
9,1,10,New Customer


# 5

In [28]:
# Calculate basic statistics for product prices by loyalty category
loyalty_spending = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

In [29]:
# Display the result
print(loyalty_spending)

                     count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                             
Loyal Customer    322011.0   9.081626  214.982149  1.0  4.2  7.3  11.3   
New Customer      195598.0  15.843383  801.060641  1.0  4.2  7.4  11.3   
Regular Customer  482391.0  11.139256  291.780563  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal Customer    99999.0  
New Customer      99999.0  
Regular Customer  99999.0  


In [32]:
# the only fluctuation is the mean and the count. other aggragation are pretty much similar

# 6

In [35]:
# Calculate the mean price of purchases per user
user_avg_spending = ords_prods_merge.groupby('user_id')['prices'].mean()

In [37]:
# Merge the user-level average spending back into the main dataset
ords_prods_merge['avg_price_per_user'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

In [39]:
# Create the spending flag
ords_prods_merge.loc[ords_prods_merge['avg_price_per_user'] < 10, 'spending_flag'] = 'Low spender'

In [41]:
ords_prods_merge.loc[ords_prods_merge['avg_price_per_user'] >= 10, 'spending_flag'] = 'High spender'

In [43]:
# Count users in each spending category
ords_prods_merge['spending_flag'].value_counts()

spending_flag
Low spender     978614
High spender     21386
Name: count, dtype: int64

# 7

In [46]:
# Calculate the median days since prior order for each user
user_median_days = ords_prods_merge.groupby('user_id')['days_since_prior_order'].median()

In [48]:
# Create a function to categorize users
def categorize_frequency(median_days):
    if median_days > 20:
        return 'Non-frequent customer'
    elif 10 < median_days <= 20:
        return 'Regular customer'
    else:
        return 'Frequent customer'

In [50]:
# Apply the function to create the flag column
ords_prods_merge['order_frequency_flag'] = ords_prods_merge['user_id'].map(user_median_days).map(categorize_frequency)

In [51]:
# Check frequency flags
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        664849
Regular customer         219649
Non-frequent customer    115502
Name: count, dtype: int64

# 8

In [55]:
# Export as pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge48.pkl'))